In [46]:
import re
import time
from time import sleep
from functools import partial
import ipywidgets as widgets
from IPython.display import display, clear_output
from itertools import product
from sympy import symbols

# Машина Тьюринга

In [47]:
#@title
class TM:
    def __init__(self):
        self.state = 'q1'
        self.pos = 0
        self.tape = []


    def print_state(self):
        print(self.state + ':', ' ' * self.pos + 'V')
        print(' ' * len(self.state) + ' ', ''.join(self.tape))


    def execute_line(self, line):
        state_A, sign_A, *other, state_B, sign_B, shift =  line.split()
        if self.state == state_A and self.tape[self.pos] == sign_A:
            self.state = state_B

            if self.pos < 0:
                self.pos = 0
                self.tape.insert(0, sign_B)
            elif self.pos >= len(self.tape):
                self.tape.append(sign_B)
            else:
                self.tape[self.pos] = sign_B

            if shift == 'L':
                self.pos -= 1
            elif shift == 'R':
                self.pos += 1

            if self.pos < 0:
                self.pos = 0
                self.tape.insert(0, '0')
            elif self.pos >= len(self.tape):
                self.tape.append('0')

            return False
        return True


    def execute(self, tape, program, timeout=0.4, verbose=True):
        self.tape = list(tape.strip('0'))
        program = tuple(line.strip() for line in program.split('\n') if line)
        while self.state != 'qz':
            if verbose: self.print_state()
            for line in program:
                executing = self.execute_line(line)
                if not executing:
                    break
            if verbose: sleep(timeout)
        if verbose: self.print_state()
        return ''.join(self.tape).strip('0')



text = widgets.Text(
    value='111*1111',
    placeholder='Tape',
    description='Tape:',
)

textarea = widgets.Textarea(
    value="""q1 * -> qz 0 E
q1 1 -> q2 0 R
q2 1 -> q2 1 R
q2 * -> qz 1 E""",
    placeholder='Program',
    description='Program:',
    disabled=False
)

run = widgets.Button(
    description='Run',
    tooltip='Run',
)

output = widgets.Output()

display(text, textarea, run, output)

def on_run_button_clicked(b):
    with output:
        clear_output(wait=True)
        prog = textarea.value
        tape = text.value
        tm = TM()
        print('\n', tm.execute(tape, prog, timeout=0.3))
        
run.on_click(on_run_button_clicked)

Text(value='111*1111', description='Tape:', placeholder='Tape')

Textarea(value='q1 * -> qz 0 E\nq1 1 -> q2 0 R\nq2 1 -> q2 1 R\nq2 * -> qz 1 E', description='Program:', place…

Button(description='Run', style=ButtonStyle(), tooltip='Run')

Output()

In [48]:
def run_tests(n, f, lim=20):
    print(f'Tests started.')
    print(f'For {n} vars. To max length {lim}.')
    prog = textarea.value
    for varables in product(range(lim + 1), repeat=n):
        tape = '*'.join('1'*v for v in varables)
        tm = TM()
        ans = tm.execute(tape, prog, timeout=0, verbose=False)
        res = len(ans)
        f_res = f(varables)
        #print(tape, ans, res, f_res)
        if res != f_res:
            print(varables)
            print('Output: ', f_res)
            print('Expected output: ', res)
            break
    else:
        print('Sucess for all tests!')


def f(v):
    x, y, z, w = v
    return y + z + 4

run_tests(4, f)

Tests started.
For 4 vars. To max length 20.
(0, 0, 0, 0)
Output:  4
Expected output:  2


# МНР

In [49]:
#@title
class MNR:
    def __init__(self, tape, prog):
        self.I = []
        self.parser(prog)
        self.R = {n: r for n, r in enumerate(tape, 1)}
        self.i = 0

    def Z(self, n):
        if n in self.R:
            self.R[n] = 0

    def S(self, n):
        if n in self.R:
            self.R[n] += 1
        else:
            self.R[n] = 1

    def T(self, m, n):
        if m in self.R:
            self.R[n] = self.R[m]
        else:
            self.R[n] = 0

    def J(self, m, n, q):
        if m in self.R and n in self.R:
            if self.R[m] == self.R[n]:
                self.i = q - 2
        elif n in self.R:
            if 0 == self.R[n]:
                self.i = q - 2
        elif m in self.R:
            if self.R[m] == 0:
                self.i = q - 2
        else:
            self.i = q - 2

    def print_field(self):
        print('I:', self.i + 1)
        print('r:', end=' ')
        for k, v in sorted(self.R.items()):
            if k == 1 or k - 1 in self.R:
                print('{: 3}'.format(v), end=' ')
            else:
                print('...', '{: 3}'.format(v), end=' ')
        print()
        print('R:', end=' ')
        for k, v in sorted(self.R.items()):
            if k == 1 or k - 1 in self.R:
                print('{: 3}'.format(k), end=' ')
            else:
                print('...', '{: 3}'.format(k), end=' ')
        print()

    def parser(self, s):
        for row in s.split('\n'):
            row = row.strip()
            if re.match(r'J\(\s*\d+\s*,\s*\d+\s*,\s*\d+\s*\)', row):
                args = map(int, re.findall(r'\d+', row))
                self.I.append(partial(self.J, *args))
            elif re.match(r'T\(\s*\d+\s*,\s*\d+\s*\)', row):
                args = map(int, re.findall(r'\d+', row))
                self.I.append(partial(self.T, *args))
            elif re.match(r'Z\(\s*\d+\s*\)', row):
                args = map(int, re.findall(r'\d+', row))
                self.I.append(partial(self.Z, *args))
            elif re.match(r'S\(\s*\d+\s*\)', row):
                args = map(int, re.findall(r'\d+', row))
                self.I.append(partial(self.S, *args))

    def run(self, timeout=0.4, verbose=True):
        while self.i < len(self.I):
            if verbose:
                self.print_field()
            time.sleep(timeout)
            self.I[self.i]()
            self.i += 1
        if verbose:
            self.print_field()
        return list(self.R.values())[0]


text = widgets.Text(
    value='4, 7',
    placeholder='Tape',
    description='Tape:',
)

textarea = widgets.Textarea(
    value="""J(2, 3, 5)
J(1, 3, 9)
S(3)
J(1, 1, 1)
Z(1)
Z(2)
Z(3)
J(1, 1, 100)
Z(1)
J(2, 3, 14)
S(1)
S(3)
J(1, 1, 10)
Z(2)
Z(3)""",
    placeholder='Program',
    description='Program:',
    disabled=False
)

run = widgets.Button(
    description='Run',
    tooltip='Run',
)

output = widgets.Output()

display(text, textarea, run, output)

def on_run_button_clicked(b):
    with output:
        clear_output(wait=True)
        prog = textarea.value
        tape = [int(i) for i in text.value.split(',')]
        m = MNR(tape, prog)
        m.run()
        
run.on_click(on_run_button_clicked)

Text(value='4, 7', description='Tape:', placeholder='Tape')

Textarea(value='J(2, 3, 5)\nJ(1, 3, 9)\nS(3)\nJ(1, 1, 1)\nZ(1)\nZ(2)\nZ(3)\nJ(1, 1, 100)\nZ(1)\nJ(2, 3, 14)\nS…

Button(description='Run', style=ButtonStyle(), tooltip='Run')

Output()

In [50]:
def run_tests(n, f, lim=20):
    print(f'Tests started.')
    print(f'For {n} vars. To max length {lim}.')
    prog = textarea.value
    for varables in product(range(lim + 1), repeat=n):
        tape = varables
        m = MNR(tape, prog)
        ans = m.run(timeout=0, verbose=False)
        res = ans
        f_res = f(varables)
        #print(tape, ans, res, f_res)
        if res != f_res:
            print(varables)
            print('Output: ', f_res)
            print('Expected output: ', res)
            break
    else:
        print('Sucess for all tests!')


def f(v):
    x, y, z, w = v
    if y > 3:
        return x + 1
    elif y < 3:
        return w + 1
    else:
        return z

run_tests(4, f)

Tests started.
For 4 vars. To max length 20.
(0, 0, 0, 0)
Output:  1
Expected output:  0


# Нормальный алгоритм Маркова

In [51]:
def instructions_parser(instr_str):
    instructions = []
    for row in instr_str.strip().replace(' ', '').split('\n'):
        instr_list = [i.strip() for i in row.split('->')]
        if len(instr_list) == 1:
            instr_list.insert(0, '')
        instructions.append(tuple(instr_list))
    return instructions


class Markov:
    def __init__(self, tape, instructions):
        self.tape = tape
        self.instructions = instructions_parser(instructions)
        self.interval = 0

    def run(self):
        while True:
            found_instr = False
            for i in self.instructions:
                to_instr = i[1].strip('.')
                if i[0] in self.tape:
                    found_instr = True
                    if i[0] == '':
                        self.tape = i[1].strip('.') + self.tape
                    else:
                        self.tape = self.tape.replace(i[0], to_instr, 1)
                    if i[1].startswith('.'):
                        return self.tape
                    break
                sleep(self.interval)
            if not found_instr:
                raise Exception('Infinite loop')


textarea = widgets.Textarea(
    value="""11a -> 1a
1a1 -> 1y
y1 -> 1y
y* -> .
a1 -> b
b1 -> b
b* -> .11
1* -> a""",
    placeholder='Program',
    description='Program:',
)

run = widgets.Button(
    description='Run',
    tooltip='Run',
)

output = widgets.Output()

instructions = textarea.value

'''
m = Markov('1111*11*1111', instructions)
res = len(m.run())
print(res)

'''

def run_tests(n, lim=50):
    print(f'Tests started.')
    print(f'For {n} vars. To max length {lim}.')
    for varables in product(range(1, lim + 1), repeat=n):
        tape = '*'.join('1'*v for v in varables)
        m = Markov(tape, instructions)
        res = len(m.run())
        f_res = f(varables)
        #print(res, f_res)
        if res != f_res:
            print(varables)
            print('Output: ', f_res)
            print('Expected output: ', res)
            break
    else:
        print('Sucess for all tests!')


n = widgets.IntText(description='Vars count:', value=3)
display(n, textarea, run, output)


"""Function for test."""
def f(v):
    x, y, z = v
    if x > 1:
        return y + z
    else:
        return z + 2


def on_run_button_clicked(b):
    with output:
        clear_output(wait=True)
        run_tests(n.value)

run.on_click(on_run_button_clicked)

IntText(value=3, description='Vars count:')

Textarea(value='11a -> 1a\n1a1 -> 1y\ny1 -> 1y\ny* -> .\na1 -> b\nb1 -> b\nb* -> .11\n1* -> a', description='P…

Button(description='Run', style=ButtonStyle(), tooltip='Run')

Output()

# Примитивная рекурсия

In [52]:
#@title
g = widgets.Text(value='x', description='g(x)')
h = widgets.Text(value='x + y', description='h(x, y, z)')
s = widgets.IntText(value='10', description='Steps:')

run = widgets.Button(
    description='Run',
    tooltip='Run',
)

output = widgets.Output()

display(g, h, s, run, output)


def calc_to_step(step=10):
    print(f'f(x, 0) = g(x) = {g.value}')
    x, y, z = symbols('x y z')
    prev_step = eval(g.value)
    for i in range(1, step + 1):
        res = eval(h.value).subs({y: i - 1, z: prev_step})
        print(f'f(x, {i}) = h(x, {i - 1}, f(x, {i - 1})) = {res}')
        prev_step = res
        

def on_run_button_clicked(b):
    with output:
        clear_output(wait=True)
        calc_to_step(s.value)
        

run.on_click(on_run_button_clicked)

Text(value='x', description='g(x)')

Text(value='x + y', description='h(x, y, z)')

IntText(value=10, description='Steps:')

Button(description='Run', style=ButtonStyle(), tooltip='Run')

Output()

# Теория нумераций

In [53]:
Pi = lambda m, n: 2**m * (2*n + 1) - 1

Dzeta = lambda m, n, q: Pi(Pi(m - 1, n - 1), q - 1)

def Tau(*a):
    res = 0
    for n, _ in enumerate(a):
        res += 2**(sum(a[:n + 1]) + n)
    return res - 1

def Beta(f, *p):
    if f == 'Z':
        return 4 * (p[0] - 1)
    elif f == 'S':
        return 4 * (p[0] - 1) + 1
    elif f == 'T':
        return 4 * Pi(p[0] - 1, p[1] - 1) + 2
    elif f == 'J':
        return 4 * Dzeta(p[0], p[1], p[2]) + 3

def C_2(x, y):
    return ((x + y)**2 + 3*x + y)/2

def C_3(x, y, z):
    return C_2(C_2(x, y), z)


In [54]:
C_2(1, 2)

7.0